<a href="https://colab.research.google.com/github/solobala/IvanGame/blob/master/Praktika_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Установка пакетов, импорт библиотек и пользовательские функции

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import time

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import SelectPercentile, chi


import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 1) Найти данные для задачи классификации или для задачи регрессии (данные не должны повторятся в группе).

In [3]:
!pip install -q kaggle # Делаем 1 раз. Датасет возьмем с kaggle

In [4]:
!kaggle -v


Kaggle API 1.7.4.5


In [5]:

# приглашение загрузить в сессионное хранилище свой kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marinabalakina","key":"0763a280a2e52ea690b44499c8ab213e"}'}

In [7]:
! mkdir ~/.kaggle # Создаем каталог .kaggle  1 раз, для того, чтобы сюда положить ключ kaggle.json, который получаем на Kaggle при регистрации


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
! cp kaggle.json ~/.kaggle/  # кладем ключ

In [9]:
! chmod 600 /root/.kaggle/kaggle.json # засекречиваем. К сожалению, это все только на время сеанса

In [10]:
!kaggle datasets list -s 'heart' # Сюда ключевое слово

ref                                                      title                                            size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  -----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
andrewmvd/heart-failure-clinical-data                    Heart Failure Prediction                         4067  2020-06-20 01:03:20.940000         175231       2451  1.0              
johnsmith88/heart-disease-dataset                        Heart Disease Dataset                            6325  2019-06-06 15:33:55.463000         290458       1576  0.88235295       
fedesoriano/heart-failure-prediction                     Heart Failure Prediction Dataset                 8762  2021-09-10 18:11:57.517000         249855       3056  1.0              
oktayrdeki/heart-disease                                 Heart Disease          

In [11]:
!kaggle datasets download -d 'fedesoriano/heart-failure-prediction'  # датасет для прогнозирования болезней сердца

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
License(s): ODbL-1.0
  0% 0.00/8.56k [00:00<?, ?B/s]
100% 8.56k/8.56k [00:00<00:00, 30.8MB/s]


In [12]:
!unzip heart-failure-prediction.zip

Archive:  heart-failure-prediction.zip
  inflating: heart.csv               


In [13]:
df = pd.read_csv('/content/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [14]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [15]:
df.info() # пропусков типа nan нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [16]:
# проверим, нет ли пропусков, обозначенных "?"
flag=True
for column in df.columns:
  if('?')  in list(df[column].value_counts()):
    print(column)
    flag = False
if flag:
    print('пропусков "?" нет')

пропусков "?" нет


In [17]:
# Категориальные features - определяем по типу object. Вместе с target их 9
cat_features = df.select_dtypes("object").columns.tolist()
print(*cat_features, sep=", ")

Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope


In [18]:
# числовые признаки
num_features = df.select_dtypes(['int', 'float']).columns.tolist()
print(*num_features, sep=", ")

Age, RestingBP, Cholesterol, FastingBS, MaxHR, Oldpeak, HeartDisease


In [19]:
# Настройка параметров оценивания алгоритма
random_seed=0
n_iter = 1000
n_estimators = 100

y = df['HeartDisease']
X = df.drop(columns=['HeartDisease'])
target_names = ['healthy', 'sick']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

Ансамблевое обучение делится на 3 основных вида:
1. Stacking. Строится n разных моделей, результаты которых являются
мета-факторами для мета-модели. То есть мета-модель – это модель,
которая обучается на результатах базовых моделей.
2. Bagging (Случайный лес). Строится n независимых друг от друга
моделей, затем, в случае решения задачи регрессии, их результаты
усредняются, в случае классификации – берется наиболее встречаемый
результат моделей.
3. Boosting (XGBoost, CatBoost, Adaboost, GBDT). Строится
последовательно n моделей, которые зависимы друг от друга.
В соответствии с заданием, реализованы ансамбли на основе баггинга и бустинга

In [41]:

# Пайплайны трансформации числовых и категориальных признаков. Числовые нормализуются, категjриальные преобразуются в 1 и 0 с помощью OneHotEncoder
num_features =['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']
numeric_transformer = Pipeline(
    steps=[ ("scaler", Normalizer())]
)


categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features),
    ]
)

base_pipelines = []
bag_pipelines = []
boost_pipelines = []

#----Пайплайн для базовой модели----------
base_pipelines.append(('SS_DTS', Pipeline([("preprocessor", preprocessor),
                                    ('DTS', DecisionTreeClassifier(random_state=random_seed))])))

#----Пайплайны для баггинга----------
bag_pipelines.append(('SS_RFC', Pipeline([("preprocessor", preprocessor),
                                     ('RFC', RandomForestClassifier(random_state=random_seed))])))

bag_pipelines.append(('SS_BGS', Pipeline([("preprocessor", preprocessor),
                                     ('BGC', BaggingClassifier(estimator=SVC(random_state=random_seed),
                        n_estimators=n_estimators, random_state=random_seed))])))

#----Пайплайны для бустинга----------
boost_pipelines.append(('SS_GBC', Pipeline([("preprocessor", preprocessor),
                                      ('GBC', GradientBoostingClassifier(random_state=random_seed))])))

boost_pipelines.append(('SS_ABC', Pipeline([("preprocessor", preprocessor),
                                      ('ABC', AdaBoostClassifier(random_state=random_seed))])))
boost_pipelines.append(('SS_LGBMC', Pipeline([("preprocessor", preprocessor),
                                      ('LGBMC', LGBMClassifier(force_col_wise=True, verbosity=-1))])))

boost_pipelines.append(('SS_XGBC', Pipeline([("preprocessor", preprocessor),
                                    ('XGBC', XGBClassifier(random_state=random_seed))])))

boost_pipelines.append(('SS_CBC', Pipeline([("preprocessor", preprocessor),
                                      ('CBC', CatBoostClassifier(random_state=random_seed,
                                         verbose=False
                                      ))])))

In [66]:
def train(X_train, X_test, y_train, y_test, pipelines) -> dict:
  # Оценивание эффективности выполнения каждого алгоритма

  names_N = []

  predictions_N = []

  results = {}
  time_in_sec = []

  for name, model in pipelines:
    start_time = time.time()
    names_N.append(name)

    m_fit = model.fit(X_train, y_train)
    m_predict = model.predict(X_test)
    execution_time = time.time() - start_time
    predictions_N.append(m_predict)

    time_in_sec.append(execution_time)

    print(f"Classification report {name}\n")
    print('====================================================================')

    print(f"{classification_report(y_test, m_predict, target_names=target_names)}\n")

    a = classification_report(y_test, m_predict, target_names=target_names, output_dict=True )['sick']
    results[f'{name}']= {"a": a, "execution_time": execution_time}
  return results

DecisionTreeClassifier() — одно дерево решений, склонно к переобучению, легко интерпретировать. Выбираем в качестве базы для сравнения (base line)

In [67]:
# базовая модель DecisionTreeClassifier
base_results = train(X_train, X_test, y_train, y_test, base_pipelines)

Classification report SS_DTS

              precision    recall  f1-score   support

     healthy       0.77      0.78      0.77        77
        sick       0.84      0.83      0.84       107

    accuracy                           0.81       184
   macro avg       0.80      0.81      0.80       184
weighted avg       0.81      0.81      0.81       184




# 2) Реализовать баггинг.



Баггинг. Строится n независимых друг от друга
моделей, затем,  в случае классификации – берется наиболее встречаемый
результат моделей.

Алгоритмы баггинга:

RandomForestClassifier() — ансамбль независимых деревьев, усредняет результаты, уменьшает переобучение

BaggingClassifier() — общий алгоритм бэггинга (с использованием SVC), параллельное обучение





In [27]:
bag_results = train(X_train, X_test, y_train, y_test, bag_pipelines)

Classification report SS_RFC

              precision    recall  f1-score   support

     healthy       0.83      0.77      0.80        77
        sick       0.84      0.89      0.86       107

    accuracy                           0.84       184
   macro avg       0.84      0.83      0.83       184
weighted avg       0.84      0.84      0.84       184


Classification report SS_BGS

              precision    recall  f1-score   support

     healthy       0.83      0.77      0.80        77
        sick       0.84      0.89      0.86       107

    accuracy                           0.84       184
   macro avg       0.84      0.83      0.83       184
weighted avg       0.84      0.84      0.84       184




# 3) Реализовать бустинг на тех же данных, что использовались для баггинга.

Бустинг. Строится
последовательно n моделей, которые зависимы друг от друга.

Алгоритмы бустинга:

GradientBoostingClassifier() — последовательное обучение деревьев на ошибках предыдущих, медленное обучение, часто высокое качество

AdaBoostClassifier() — бустинг с увеличением веса ошибочных объектов, чувствителен к выбросам

XGBClassifier() — градиентный бустинг с регуляризацией, растет level-wise, надежный универсальный алгоритм

CatBoostClassifier() — бустинг с автоматической обработкой категориальных признаков, устойчив к переобучению

In [42]:
boost_results = train(X_train, X_test, y_train, y_test, boost_pipelines)

Classification report SS_GBC

              precision    recall  f1-score   support

     healthy       0.82      0.82      0.82        77
        sick       0.87      0.87      0.87       107

    accuracy                           0.85       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.85      0.85      0.85       184


Classification report SS_ABC

              precision    recall  f1-score   support

     healthy       0.83      0.78      0.81        77
        sick       0.85      0.89      0.87       107

    accuracy                           0.84       184
   macro avg       0.84      0.83      0.84       184
weighted avg       0.84      0.84      0.84       184


Classification report SS_LGBMC

              precision    recall  f1-score   support

     healthy       0.80      0.84      0.82        77
        sick       0.88      0.85      0.87       107

    accuracy                           0.85       184
   macro avg       0.84      0.85 

# 4) Сравнить результаты работы алгоритмов (время работы и качество
моделей). Сделать выводы.

In [43]:
def view_results(results):
  df = pd.DataFrame([
      {
          'model_name': name,
          **data['a'],
          'execution_time': data['execution_time']
      }
      for name, data in results.items()
  ]).rename(columns={'precision': 'precision', 'recall': 'recall', 'f1-score': 'f1-score'})
  df.drop('support', axis=1, inplace=True)
  return df


In [44]:
base=view_results(base_results)

In [45]:
numeric_cols = base.select_dtypes(include=[np.number]).columns
base.head().style.format('{:.4f}', subset=numeric_cols, na_rep='-').background_gradient(cmap='Blues', subset=numeric_cols)

,model_name,precision,recall,f1-score,execution_time
0,SS_DTS,0.8396,0.8318,0.8357,0.0239


In [55]:
bags=view_results(bag_results)
bags.head().style.format('{:.4f}', subset=numeric_cols, na_rep='-').background_gradient(cmap='Blues', subset=numeric_cols.drop('execution_time')).background_gradient(cmap='Blues_r', subset=['execution_time'])

,model_name,precision,recall,f1-score,execution_time
0,SS_RFC,0.8407,0.8879,0.8636,0.3372
1,SS_BGS,0.8407,0.8879,0.8636,0.8463


In [56]:
boosts=view_results(boost_results)
(boosts.head()
          .style
          .format('{:.4f}', subset=numeric_cols, na_rep='-')
          .background_gradient(cmap='Blues', subset=numeric_cols.drop('execution_time'))
          .background_gradient(cmap='Blues_r', subset=['execution_time']))


,model_name,precision,recall,f1-score,execution_time
0,SS_GBC,0.8692,0.8692,0.8692,0.3670
1,SS_ABC,0.8482,0.8879,0.8676,0.1297
2,SS_LGBMC,0.8835,0.8505,0.8667,0.0667
3,SS_XGBC,0.8679,0.8598,0.8638,0.0732
4,SS_CBC,0.8679,0.8598,0.8638,1.7298


# 5) Оформить отчет о проделанной работе.

Выводы типа такие:
во-первых, нам важнее, чтобы хорошо прогнозировались болезни сердца, а не его здоровье, поэтому метрики важны для 'sick', а не для healthy, их и берем для рассмотрения
во-вторых, рассмотрели три вида моделей - одиночную, и 2 типа ансамблей - на оснолвем бэггинга и бустинга. Общий вывод: все модели отработали хорошо, показав и precision, и recall более 83%.
Вместе с тем, ансамбли помогают дополнительно улучшить мептрики в сравнении с отдельной одиночной моделью, но время обучения растет.
НАпример, метрики при бэггинге возросли незначительно по сравнению с одиночной моделью, а время выполнения увеличилось белее чем в 15 раз. Бустинговые модели показали более существенный  рост всех метрик. Лучше всего сработал 	SS_LGBMC - у него время обучения сравнимо с одиночной моделью, а метрики выше.